# Using _egobox_ optimizer _Egor_

## Installation

In [1]:
%pip install egobox

Note: you may need to restart the kernel to use updated packages.


We import _egobox_ as _egx_ for short.

In [2]:
import numpy as np
import egobox as egx

You may setup the logging level to get optimization progress during the execution

In [3]:
# To display optimization information (none by default)
# import logging
# logging.basicConfig(level=logging.INFO)

## Example 1 : Continuous optimization

### Test functions

In [4]:
xspecs_xsinx = egx.to_specs([[0., 25.]])
n_cstr_xsinx = 0

def xsinx(x: np.ndarray) -> np.ndarray:
    x = np.atleast_2d(x)
    y = (x - 3.5) * np.sin((x - 3.5) / (np.pi))
    return y

In [5]:
xspecs_g24 = egx.to_specs([[0., 3.], [0., 4.]])
n_cstr_g24 = 2

# Objective
def G24(point):
    """
    Function g24
    1 global optimum y_opt = -5.5080 at x_opt =(2.3295, 3.1785)
    """
    p = np.atleast_2d(point)
    return - p[:, 0] - p[:, 1]

# Constraints < 0
def G24_c1(point):
    p = np.atleast_2d(point)
    return (- 2.0 * p[:, 0] ** 4.0
            + 8.0 * p[:, 0] ** 3.0 
            - 8.0 * p[:, 0] ** 2.0 
            + p[:, 1] - 2.0)

def G24_c2(point):
    p = np.atleast_2d(point)
    return (-4.0 * p[:, 0] ** 4.0
            + 32.0 * p[:, 0] ** 3.0
            - 88.0 * p[:, 0] ** 2.0
            + 96.0 * p[:, 0]
            + p[:, 1] - 36.0)

# Grouped evaluation
def g24(point):
    p = np.atleast_2d(point)
    return np.array([G24(p), G24_c1(p), G24_c2(p)]).T


### Continuous optimization

In [6]:
egor = egx.Egor(xspecs_xsinx, n_cstr=n_cstr_xsinx)   # see help(egor) for options

In [7]:
res = egor.minimize(xsinx, max_iters=8)
print(f"Optimization f={res.y_opt} at {res.x_opt}")
print("Optimization history: ")
print(f"Inputs = {res.x_hist}")
print(f"Outputs = {res.y_hist}")

Optimization f=[-15.12510323] at [18.93509877]
Optimization history: 
Inputs = [[24.16632486]
 [11.46890607]
 [18.44757257]
 [ 7.79926692]
 [ 2.17759007]
 [18.12777788]
 [18.68945993]
 [18.9361961 ]
 [18.93481137]
 [18.93489165]
 [18.93509877]
 [18.93564231]
 [18.93490956]]
Outputs = [[  6.01070335]
 [  4.53248411]
 [-14.93205423]
 [  4.21159462]
 [  0.54035662]
 [-14.60466485]
 [-15.07551091]
 [-15.12510243]
 [-15.1251031 ]
 [-15.12510315]
 [-15.12510323]
 [-15.12510308]
 [-15.12510316]]


In [8]:
egor = egx.Egor(xspecs_g24, 
                n_doe=10, 
                n_cstr=n_cstr_g24, 
                cstr_tol=[1e-3, 1e-3],
                infill_strategy=egx.InfillStrategy.WB2,
                target=-5.5,
                # outdir="./out",
                # hot_start=True
            )   

# Specify regression and/or correlation models used to build the surrogates of objective and constraints
#egor = egx.Egor(g24, xlimits_g24, n_cstr=n_cstr_g24, n_doe=10,
#                      regr_spec=egx.RegressionSpec.LINEAR,
#                      corr_spec=egx.CorrelationSpec.MATERN32 | egx.CorrelationSpec.MATERN52)  

In [9]:
res = egor.minimize(g24, max_iters=30)
print(f"Optimization f={res.y_opt} at {res.x_opt}")
print("Optimization history: ")
print(f"Inputs = {res.x_hist}")
print(f"Outputs = {res.y_hist}")

Optimization f=[-5.50857874e+00 -1.02345305e-04  8.35057110e-04] at [2.32959306 3.17898568]
Optimization history: 
Inputs = [[2.02750492 1.84783549]
 [2.43770908 0.79253687]
 [2.83077386 3.33954312]
 [1.61764313 0.13863444]
 [0.42165631 2.71057275]
 [1.22945986 1.04947757]
 [1.12529889 3.10581662]
 [0.62145558 3.60508041]
 [0.0868541  2.01094979]
 [2.26033054 1.3874575 ]
 [2.15407992 3.95417039]
 [2.3260228  3.1921383 ]
 [2.32959306 3.17898568]]
Outputs = [[-3.87534041e+00 -1.58384279e-01 -2.14611463e+00]
 [-3.23024595e+00 -3.48447274e+00 -1.82157495e+00]
 [-6.17031698e+00 -9.72175238e+00  2.95560230e+00]
 [-1.75627757e+00 -2.62649202e+00 -2.77728535e+00]
 [-3.13222905e+00 -1.75257198e-01 -6.18376493e+00]
 [-2.27893743e+00 -2.74545955e+00  3.89263155e-01]
 [-4.23111551e+00 -8.31877099e-01  2.88510841e+00]
 [-4.22653599e+00  1.37196496e-01  3.62309923e-01]
 [-2.09780388e+00 -4.42715310e-02 -2.62941582e+01]
 [-3.64778804e+00 -1.30504948e+00 -2.08873873e+00]
 [-6.10825031e+00  1.73385454e

### Egor as a service: ask-and-tell interface

When the user needs to be in control of the optimization loop, `Egor` can be used as a service. 

For instance with the `xsinx` objective function, we can do:

In [10]:
xlimits = egx.to_specs([[0.0, 25.0]])
egor = egx.Egor(xlimits, seed=42) 

# initial doe
x_doe = egx.lhs(xlimits, 3, seed=42)
y_doe = xsinx(x_doe)
for _ in range(10): # run for 10 iterations
    x = egor.suggest(x_doe, y_doe)  # ask for best location
    x_doe = np.concatenate((x_doe, x))
    y_doe = np.concatenate((y_doe, xsinx(x))) 
res = egor.get_result(x_doe, y_doe)
print(f"Optimization f={res.y_opt} at {res.x_opt}")

Optimization f=[-15.12510323] at [18.93528147]


## Example 2 : Mixed-integer optimization

### Test function

In [11]:
xspecs_mixint_xsinx = [egx.XSpec(egx.XType.INT, [0, 25])]
n_cstr_mixint_xsinx = 0

def mixint_xsinx(x: np.ndarray) -> np.ndarray:
    x = np.atleast_2d(x)
    if (np.abs(np.linalg.norm(np.floor(x))-np.linalg.norm(x))< 1e-8):
        y = (x - 3.5) * np.sin((x - 3.5) / (np.pi))
    else:
        raise ValueError(f"Bad input: mixint_xsinx accepts integer only, got {x}")
    return y

### Mixed-integer optimization with _Egor_

In [12]:
egor = egx.Egor(xspecs_mixint_xsinx, 
                     n_doe=3, 
                     infill_strategy=egx.InfillStrategy.EI,
                     target=-15.12,
                    )  # see help(egor) for options
res = egor.minimize(mixint_xsinx, max_iters=30)
print(f"Optimization f={res.y_opt} at {res.x_opt}")
print("Optimization history: ")
print(f"Inputs = {res.x_hist}")
print(f"Outputs = {res.y_hist}")

Optimization f=[-15.12161154] at [19.]
Optimization history: 
Inputs = [[23.]
 [ 8.]
 [ 9.]
 [20.]
 [18.]
 [19.]]
Outputs = [[ -1.48334497]
 [  4.45696985]
 [  5.41123083]
 [-14.15453288]
 [-14.43198471]
 [-15.12161154]]


## Example 3 : More mixed-integer optimization

In the following example we see we can have other special integer type cases, where a component of x can take one value out of a list of ordered values (ORD type) or being like an enum value (ENUM type). Those types differ by the processing related to the continuous relaxation made behind the scene:
* For INT type, resulting float is rounded to the closest int value,
* For ORD type, resulting float is cast to closest value among the given valid ones,
* For ENUM type, one hot encoding is performed to give the resulting value.  

### Test function

In [13]:
# Objective function which takes [FLOAT, ENUM1, ENUM2, ORD] as input
# Note that ENUM values are passed as indice value eg either 0, 1 or 2 for a 3-sized enum  
def mixobj(X):
    # float
    x1 = X[:, 0]
    #  ENUM 1
    c1 = X[:, 1]
    x2 = c1 == 0
    x3 = c1 == 1
    x4 = c1 == 2
    #  ENUM 2
    c2 = X[:, 2]
    x5 = c2 == 0
    x6 = c2 == 1
    # int
    i = X[:, 3]

    y = (x2 + 2 * x3 + 3 * x4) * x5 * x1 + (x2 + 2 * x3 + 3 * x4) * x6 * 0.95 * x1 + i
    return y.reshape(-1, 1)

### Mixed-integer optimization with _Egor_

In [14]:
xtypes = [
    egx.XSpec(egx.XType.FLOAT, [-5.0, 5.0]),
    egx.XSpec(egx.XType.ENUM, tags=["blue", "red", "green"]),
    egx.XSpec(egx.XType.ENUM, xlimits=[2]),
    egx.XSpec(egx.XType.ORD, [0, 2, 3]),
]
egor = egx.Egor(xtypes, seed=42)
res = egor.minimize(mixobj, max_iters=10)
print(f"Optimization f={res.y_opt} at {res.x_opt}")
print("Optimization history: ")
print(f"Inputs = {res.x_hist}")
print(f"Outputs = {res.y_hist}")

Optimization f=[-14.25] at [-5.  2.  1.  0.]
Optimization history: 
Inputs = [[-1.90197486  2.          1.          3.        ]
 [ 1.36933896  1.          0.          2.        ]
 [-0.10843099  1.          0.          0.        ]
 [-4.73477511  0.          0.          3.        ]
 [ 3.11266243  2.          1.          2.        ]
 [ 0.33069418  2.          1.          0.        ]
 [ 4.47594664  2.          1.          0.        ]
 [-3.26619512  0.          0.          2.        ]
 [-5.          2.          1.          2.        ]
 [-5.          2.          1.          0.        ]
 [-5.          2.          1.          0.        ]
 [-5.          2.          1.          0.        ]
 [-5.          2.          1.          0.        ]
 [-5.          1.          0.          0.        ]
 [-5.          2.          1.          0.        ]
 [-5.          2.          1.          0.        ]
 [-5.          2.          1.          0.        ]
 [-5.          2.          1.          0.        ]]
Outp

Note that `x_opt` result contains indices for corresponding optional tags list hence the second component should be read as 0="red", 1="green", 2="blue", while the third component was unamed 0 correspond to first enum value and 1 to the second one.

## Usage

In [15]:
help(egor)

Help on Egor in module builtins object:

class Egor(object)
 |  Egor(xspecs, n_cstr=0, cstr_tol=None, n_start=20, n_doe=0, doe=None, regr_spec=Ellipsis, corr_spec=Ellipsis, infill_strategy=Ellipsis, q_points=1, par_infill_strategy=Ellipsis, infill_optimizer=Ellipsis, kpls_dim=None, n_clusters=1, target=Ellipsis, outdir=None, hot_start=False, seed=None)
 |  
 |  Optimizer constructor
 |  
 |  fun: array[n, nx]) -> array[n, ny]
 |       the function to be minimized
 |       fun(x) = [obj(x), cstr_1(x), ... cstr_k(x)] where
 |          obj is the objective function [n, nx] -> [n, 1]
 |          cstr_i is the ith constraint function [n, nx] -> [n, 1]
 |          an k the number of constraints (n_cstr)
 |          hence ny = 1 (obj) + k (cstrs)
 |       cstr functions are expected be negative (<=0) at the optimum.
 |  
 |   n_cstr (int):
 |       the number of constraint functions.
 |  
 |   cstr_tol (list(n_cstr,)):
 |       List of tolerances for constraints to be satisfied (cstr < tol), 